# setup

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
print("sklearn version: ", sklearn.__version__)
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
print("TF version: ", tf.__version__)
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# GPU test
print("GPU installed: ",tf.test.is_built_with_gpu_support())

# To prevent "CUDNN_STATUS_ALLOC_FAILED" error with GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")    

from tensorflow.python.keras.utils.data_utils import Sequence

sklearn version:  1.0.2
TF version:  2.8.0
No GPU was detected. CNNs can be very slow without a GPU.
GPU installed:  True


# Get the data

In [ ]:
#작성자 전민재
import numpy as np
import csv
#"./emnist-byclass-test.csv"
#train
csv_data_file = open("../공용/train_data.csv", "r", encoding="ms932", errors="", newline="" ) #리스트 형식 
csv_label_file = open("../공용/train_label.csv", "r", encoding="ms932", errors="", newline="" ) #리스트 형식 

#valid
#csv_data_file = open("./valid_data.csv", "r", encoding="ms932", errors="", newline="" ) #리스트 형식 
#csv_label_file = open("./valid_label.csv", "r", encoding="ms932", errors="", newline="" ) #리스트 형식 
f = csv.reader(csv_data_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
l=csv.reader(csv_label_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
X_train=[]
y_train=[]
X_valid=[]
y_valid=[]

for i, row in enumerate(f):
    #행마다 int로 형변환
    for idx, char in enumerate(row):
        row[idx]=int(char)
    #train
    #data 추가    
    X_train.append(row)
    
    #valid
    #data 추가    
    # X_valid.append(row)
        

for i, row in enumerate(l):
    
    #train
    #label 추가
    y_train.append(int(row[0]))
    
    #valid
    #label 추가
    # y_valid.append(int(row[0]))
        
#train
X_train=np.array(X_train)
X_train=X_train.reshape(-1,28,28)

#valid
# X_valid=np.array(X_valid)
# X_valid=X_valid.reshape(-1,28,28)





# SMV2

In [2]:
class Inveted_Residual_Block(keras.layers.Layer):
    def __init__(self,c,s=1,t=1,n=1,**kwargs):
        super().__init__(**kwargs)
        self.activation = keras.layers.ReLU(max_value=6)
        self.main_layers = [
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=c*t,kernel_size=1,strides=1,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3,strides=1,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=c,kernel_size=1,strides=1,padding="SAME",activation=self.activation),
            ]
        self.strides_layers=[
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=c*t,kernel_size=1,strides=1,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3,strides=s,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=c,kernel_size=1,strides=1,padding="SAME",activation=self.activation),
        ]
        self.s=s
        self.t=t
        self.c=c
        self.n=n

    def call(self, inputs):
        Z = inputs
        
        for n in range(0,self.n):
            if n == 0:
                for layer in self.strides_layers:
                    Z = layer(Z)
            else:
                skip_Z = Z
                for layer in self.main_layers:
                    Z = layer(Z)
                
                Z += skip_Z
    
        return Z
    
    def get_config(self):
        base=super().get_config()
        return{**base,"s":self.s,"t":self.t,"c":self.c,"n":self.n}

In [3]:
smv2p=keras.models.load_model("smv2p_nadam.h5",
                               custom_objects={"Inveted_Residual_Block" : Inveted_Residual_Block})


OSError: No file or directory found at smv2p_nadam.h5